# Version Control with Icechunk

In [1]:
import zarr
from icechunk import IcechunkStore, StorageConfig

## Create a new Zarr store backed by Icechunk

This example uses an in-memory store.

In [2]:
store = IcechunkStore.create(
    storage=StorageConfig.memory("test")
)
store

1. Why not checkout main by default?
2. Why can I create snapshots on the `None` branch

## Snaphotting

### Concepts

1. `store.commit` creates a _snapshot_ of the data.
2. Every snapshot is associated with a _snapshot ID_.
3. Use the _snapshot ID_ to time-travel within your data's history.

### Create a snapshot

In [3]:
root_group = zarr.group(store=store)

In [4]:
root_group.attrs["attr"] = "first_attr"

In [5]:
first_commit = store.commit("first commit")
first_commit

'51MXCR5RTNGPC54Z7WJG'

In [6]:
dict(root_group.attrs)

{'attr': 'first_attr'}

In [7]:
root_group.attrs["attr"] = "second_attr"
second_commit = store.commit("second commit")
second_commit

'45AE3AT46RHZCZ50HWEG'

### View the current snapshot ID

In [8]:
store.snapshot_id

'45AE3AT46RHZCZ50HWEG'

### Time-travel to a snapshot

Here's where we are:

In [9]:
store.snapshot_id, dict(root_group.attrs)

('45AE3AT46RHZCZ50HWEG', {'attr': 'second_attr'})

In [10]:
store.checkout(snapshot_id=first_commit)
root_group = zarr.group(store=store)
dict(root_group.attrs)

{'attr': 'first_attr'}

### Snapshotting is only allowed at the tip of a branch

TODO: need better error message

In [11]:
root_group.attrs["attr"] = "will_fail"
store.commit("this should fail")

ValueError: store error: all commits must be made on a branch

## Branching


### View the current branch

TODO: why is this None

In [12]:
store.branch

### Create a new branch

We will create a new branch starting at `first_commit`

In [13]:
store.reset()

In [14]:
assert store.snapshot_id == first_commit

In [15]:
store.new_branch("new-branch")

'51MXCR5RTNGPC54Z7WJG'

In [16]:
store.branch

'new-branch'

In [17]:
root_group = zarr.group(store=store)
dict(root_group.attrs)

{'attr': 'first_attr'}

In [18]:
root_group.attrs["attr"] = "new_branch_attr"
new_branch_commit = store.commit("commit on new branch")

### Switch branches

Use the `branch` argument of `checkout` to switch to a different branch

In [19]:
store.checkout(branch="main")

In [20]:
store.snapshot_id == second_commit

True

In [21]:
store.checkout(branch="new-branch")
store.snapshot_id == new_branch_commit

True

## Tagging

### Creating a new tag

In [22]:
store.checkout(branch="main")

TODO: use current snapshot_id by default?

In [23]:
store.tag("v0", snapshot_id=store.snapshot_id)

In [24]:
store.tag("v-1", snapshot_id=first_commit)

### Time-travel to a tag

Pass the `tag` argument to `checkout`

In [25]:
store.checkout(tag="v-1")
store.snapshot_id == first_commit

True

In [26]:
store.checkout(branch="main")